In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import meteva.base as meb
import numpy as np
import datetime

为了方便在程序中共享网格设置，本模块中将网格信息封装成一个grid类变量。  
<font face="黑体" color=blue size=5>网格信息类</font>

**class grid**  
 \_\_init\_\_(self,glon, glat, gtime=None, dtime_list=None,level_list=None,member_list = None)     

**参数说明：**  
  glon : 经向网格信息：[起始经度，终止经度，经向间隔]  
  glat : 纬向网格信息：[起始纬度，终止纬度，纬向间隔]  
  gtime : 时间网格信息：若gtime只包含1个元  素，其内容为datetime变量，或者常用字符形式时间，例如"2019123108"、"2019-12-31:08"或"19年12月31日08时"等方式都能兼容。若gtime包含三个元素，其内容为[起始时间,结束时间,时间间隔], 其中时间格式可以是datetime变量或者字符串形式，时间间隔为字符串，例如"12h"代表12小时，"30m"代表30分钟。    
  dtime_list : 时效列表,其元素dtime为整数，若取值小于10000，则dtime的单位为小时，若取值大10000，则  时效 = (dtime - 10000)分钟  
  level_list : 层次名称列表  
  member_list : -成员名称列表  

**其初始化方式如以下示例**

In [2]:
grid0 = meb.grid([100,110,1],[22,20,-1])  #初始化一个网格，其中时间、时效、层次和成员信息都暂时使用缺省设置
print(grid0)

members:['data0']
levels:[0]
gtime:['20990101000000', '20990101000000', '1h']
dtimes:[0]
glon:[100, 110.0, 1]
glat:[22, 20.0, -1]



In [3]:
grid0 = meb.grid([100,110,1],[22,20,-1],gtime=["2019013108","2019020108","24h"],dtime_list = [24],level_list = [0],member_list = ["GRAPES"])
print(grid0)

members:['GRAPES']
levels:[0]
gtime:['20190131080000', '20190201080000', '24h']
dtimes:[24]
glon:[100, 110.0, 1]
glat:[22, 20.0, -1]



<font face="黑体" color=blue size=5>网格数据类</font>  
格式为[xarray.DataArray](http://xarray.pydata.org/en/stable/data-structures.html)的数据。本函数库提供了对该网格数据的一序列初始化、读写和转换函数，经过本函数库初始化、读取或转换生成的网格数据统一具备顺序为member，level，time，dtime，lat，lon 共6个维度的坐标，其中水平方向为等经纬度坐标。

### 初始化网格数据
**grid_data（grid,data=None）**  
根据网格信息初始化一个网格数据

**参数说明：**  
 grid: 网格信息类变量  
 data: numpy数组，缺省时生成内容全为0的网格数据  
 return: xarray.DataArray格式数据，其中依次包含member，level，time，dtime，lat，lon共6个维度的信息。


**调用示例**

In [4]:
grd = meb.grid_data(grid0)  #根据网格信息生成网格数据，网格值都为0
print(grd)

<xarray.DataArray 'data0' (member: 1, level: 1, time: 2, dtime: 1, lat: 3, lon: 11)>
array([[[[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],


         [[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]]]])
Coordinates:
  * member   (member) <U6 'GRAPES'
  * level    (level) int32 0
  * time     (time) datetime64[ns] 2019-01-31T08:00:00 2019-02-01T08:00:00
  * dtime    (dtime) int32 24
  * lat      (lat) int32 22 21 20
  * lon      (lon) int32 100 101 102 103 104 105 106 107 108 109 110


In [5]:
x= np.arange(11)
y= np.arange(3)
xx,yy = np.meshgrid(x,y)
dat = np.array([xx,yy])
grd = meb.grid_data(grid0,dat)   #根据网格信息和numpy数组生成网格数
print(grd)

<xarray.DataArray 'data0' (member: 1, level: 1, time: 2, dtime: 1, lat: 3, lon: 11)>
array([[[[[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]]],


         [[[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
           [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
           [ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2]]]]]])
Coordinates:
  * member   (member) <U6 'GRAPES'
  * level    (level) int32 0
  * time     (time) datetime64[ns] 2019-01-31T08:00:00 2019-02-01T08:00:00
  * dtime    (dtime) int32 24
  * lat      (lat) int32 22 21 20
  * lon      (lon) int32 100 101 102 103 104 105 106 107 108 109 110


### 重置网格数据
**reset（grd）**  
将网格的经向坐标调整为由西向东递增，由南向北递增  

**参数说明：**  
 grd: 网格数据  
 return: None


**调用示例**

In [6]:
meb.reset(grd)
print(grd)  #重置后坐标及相应的数据都会相应调整


<xarray.DataArray 'data0' (member: 1, level: 1, time: 2, dtime: 1, lat: 3, lon: 11)>
array([[[[[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]]],


         [[[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
           [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
           [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]]]]]])
Coordinates:
  * member   (member) <U6 'GRAPES'
  * level    (level) int32 0
  * time     (time) datetime64[ns] 2019-01-31T08:00:00 2019-02-01T08:00:00
  * dtime    (dtime) int32 24
  * lat      (lat) int32 20 21 22
  * lon      (lon) int32 100 101 102 103 104 105 106 107 108 109 110


In [7]:
grid1 = meb.get_grid_of_data(grd) 
print(grid1)  #打印重置后的网格信息

members:['GRAPES']
levels:[0]
gtime:['20190131080000', '20190201080000', '24h']
dtimes:[24]
glon:[100, 110.0, 1]
glat:[20, 22.0, 1]



### 设置网格的坐标属性
**set_griddata_coords(grd,name = None,gtime = None,dtime_list = None, level_list = None, member_list = None):**  
对网格的维度坐标属性进行设置  

**参数说明：**  
 **grd**: 网格数据
 **name**: 数据内容名称，缺省时不做改变  
 **level_list**: 层次坐标列表，缺省时不做改变    
 **gtime**: 时间坐标参数列表，它可以包括3个或1个元素。当grid_data的time维度size=1时，gtime只包含1个元素，其内容为datetime变量，或者常用字符形式时间，例如"2019123108"、"2019-12-31:08"或"19年12月31日08时"等方式都能兼容。如果grid_data的time维度size>1，则gtime包含三个元素其内容为，分别为[起始时间,结束时间,时间间隔], 其中时间格式可以是datetime变量或者字符串形式，时间间隔为字符串，例如"12h"代表12小时，"30m"代表30分钟。，缺省时不做改变    
 **gtime_list**: 时效坐标列表,其元素dtime为整数，若取值小于10000，则时效 = dtime小时，若取值大10000，则代表 时效 = (dtime - 10000)分钟，缺省时不做改变    
 **member_list**: 成员名称列表，缺省时不做改变    
 return: None  


**调用示例**

In [9]:
meb.set_griddata_coords(grd,name = "Temperature",level_list = [0],gtime=["2019-01-31:08"],dtime_list = [24],member_list = ["GRAPES"])
print(grd)

gtime对应的时间序列长度和grid_data的time维度的长度不一致
<xarray.DataArray 'Temperature' (member: 1, level: 1, time: 2, dtime: 1, lat: 3, lon: 11)>
array([[[[[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]]],


         [[[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
           [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
           [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]]]]]])
Coordinates:
  * member   (member) <U6 'GRAPES'
  * level    (level) int32 0
  * time     (time) datetime64[ns] 2019-01-31T08:00:00 2019-02-01T08:00:00
  * dtime    (dtime) int32 24
  * lat      (lat) int32 20 21 22
  * lon      (lon) int32 100 101 102 103 104 105 106 107 108 109 110


In [11]:
time1 = datetime.datetime(2019,2,1,8,0)
meb.set_griddata_coords(grd,gtime=[time1])
print(grd)

gtime对应的时间序列长度和grid_data的time维度的长度不一致
<xarray.DataArray 'Temperature' (member: 1, level: 1, time: 2, dtime: 1, lat: 3, lon: 11)>
array([[[[[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]]],


         [[[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
           [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
           [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]]]]]])
Coordinates:
  * member   (member) <U6 'GRAPES'
  * level    (level) int32 0
  * time     (time) datetime64[ns] 2019-01-31T08:00:00 2019-02-01T08:00:00
  * dtime    (dtime) int32 24
  * lat      (lat) int32 20 21 22
  * lon      (lon) int32 100 101 102 103 104 105 106 107 108 109 110
